In [1]:
# Import libraries
import pandas as pd
import os
from pathlib import Path
from tqdm import tqdm
import yaml
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

In [2]:
LEARNED_THR = 0.
IOU_THR = 0.6
PREDICTION_PATH = "dataset/predictions/04-predictions.csv"

In [3]:
PREDICTION_IMAGE_PATH = Path("dataset/images/test")

test_df = pd.read_csv(PREDICTION_PATH).rename(
    columns={"x_min": "xmin", "y_min": "ymin", "x_max": "xmax", "y_max": "ymax"}
)
test_df.sample(5)

,Image_ID,confidence,class_id,class,xmin,ymin,xmax,ymax
17783,ID_CjxpRk.JPG,0.086723,0,anthracnose,1525.413574,117.790321,2413.988281,2787.230225
15333,ID_hLLkip.jpg,0.737704,2,healthy,878.841675,1169.795776,1325.228882,1948.375366
10731,ID_iPhQrE.jpg,0.238383,1,cssvd,592.755798,33.074036,932.445374,1061.696533
11552,ID_CRgjeZ.jpg,0.084483,2,healthy,14.779973,1014.721313,348.219696,1781.357300
8810,ID_EmaVCt.jpg,0.181484,2,healthy,29.523699,9.237042,140.938828,121.371651


In [4]:
class_id = {j: i for i, j in enumerate(sorted(test_df["class"].unique()))}
test_df["class_id"] = test_df["class"].map(class_id)
class_id

{'anthracnose': 0, 'cssvd': 1, 'healthy': 2}

In [5]:
test_df.head()

,Image_ID,confidence,class_id,class,xmin,ymin,xmax,ymax
0,ID_cWEAQI.jpeg,0.432404,2,healthy,-15.221375,56.677937,1572.595459,3975.569824
1,ID_cWEAQI.jpeg,0.143457,2,healthy,1410.376099,322.058899,1739.138550,1089.723755
2,ID_cWEAQI.jpeg,0.088023,2,healthy,13.658841,148.699219,449.274506,1109.639893
3,ID_cWEAQI.jpeg,0.078814,2,healthy,1120.934204,14.121279,1699.360962,474.832306
4,ID_cWEAQI.jpeg,0.065425,2,healthy,0.516278,756.302246,213.454803,1553.445190


In [6]:
import torch

def convert_df(df: pd.DataFrame):
	df = df.copy().dropna()
	return {
		img_id: {
			"boxes": torch.tensor(raw[["xmin", "ymin", "xmax", "ymax"]].values, dtype=torch.float32),
			"scores": (
				torch.tensor(raw["confidence"].values, dtype=torch.float32)
				if "confidence" in raw.columns
				else None
			),
			"labels": torch.tensor(raw["class_id"].values, dtype=torch.int32),
		}
		for (img_id, ), raw in df.groupby(["Image_ID"])
	}

def default_value():
	return {
		"boxes": torch.empty((0, 4), dtype=torch.float32),
		"scores": torch.empty((0,), dtype=torch.float32),
		"labels": torch.empty((0,), dtype=torch.int32),
	}


converted_labels = list(PREDICTION_IMAGE_PATH.glob("*"))
converted_labels = [i.name for i in converted_labels]

def get_preds_data(preds, thr: float = 0.5):
	if thr is not None:
		preds = preds[preds["confidence"] >= thr]
	preds = convert_df(preds)
	print(len(preds))
	print(list(preds.keys())[:10])
	d = default_value()
	return {i: preds.get(i, d) for i in converted_labels}

In [7]:
test_df.sample(2)

,Image_ID,confidence,class_id,class,xmin,ymin,xmax,ymax
7203,ID_qQRBL9.jpeg,0.452241,2,healthy,-3.560677,80.323433,1089.573975,513.825684
16031,ID_HiEYk8.jpg,0.115299,2,healthy,0.379518,1341.371338,634.127930,1834.378784


In [8]:
predictions = get_preds_data(test_df, thr=None)

len(predictions)

1626
['ID_A16nzu.jpg', 'ID_A1Euyz.jpg', 'ID_A1HcV0.jpeg', 'ID_A4ZdJC.jpeg', 'ID_A5SFUW.jpeg', 'ID_A6Fogi.jpeg', 'ID_ABDCyn.jpeg', 'ID_ACg6Qf.jpeg', 'ID_AFi8A1.jpg', 'ID_AHlc9P.jpg']


1626

In [9]:
predictions["ID_CGnAYP.jpg"]

{'boxes': tensor([[   3.4422,   22.8539,  700.2789, 1273.0865],
         [ 605.8710,   45.9294,  940.0273, 1141.3771],
         [ 455.4386,   26.7915,  936.0859,  337.5445],
         [ 616.6327,  285.8711,  829.3041,  692.5198],
         [  60.3104,   16.0185,  544.8130,  435.8484]]),
 'scores': tensor([0.6946, 0.4088, 0.1437, 0.0652, 0.0613]),
 'labels': tensor([1, 1, 1, 1, 1], dtype=torch.int32)}

In [10]:
predictions["ID_CGnAYP.jpg"].keys()

dict_keys(['boxes', 'scores', 'labels'])

In [11]:
from torchvision.ops import nms

def apply_nms(predictions: dict[str, torch.Tensor], iou_threshold=0.5):
    """
    Apply Non-Maximum Suppression (NMS) to reduce overlapping boxes.

    Args:
        predictions (dict): Dictionary containing keys ['boxes', 'scores', 'labels'].
        iou_threshold (float): Intersection over Union (IoU) threshold for NMS.

    Returns:
        dict: Filtered predictions after applying NMS.
    """
    filtered_predictions = {}
    for image_id, data in predictions.items():
        boxes = data["boxes"]
        scores = data["scores"]
        labels = data["labels"]

        if boxes.numel() == 0:
            # If no boxes, skip this image
            filtered_predictions[image_id] = data
            continue

        # Perform NMS for each class separately
        keep_indices = []
        for label in labels.unique():
            label_mask = labels == label
            label_boxes = boxes[label_mask]
            label_scores = scores[label_mask]
            indices = nms(label_boxes, label_scores, iou_threshold)
            keep_indices.extend(label_mask.nonzero(as_tuple=True)[0][indices].tolist())

        # Filter boxes, scores, and labels based on NMS results
        keep_indices = torch.tensor(keep_indices, dtype=torch.long)
        filtered_predictions[image_id] = {
            "boxes": boxes[keep_indices],
            "scores": scores[keep_indices],
            "labels": labels[keep_indices],
        }

    return filtered_predictions

# Apply NMS to predictions
filtered_predictions = apply_nms(predictions, iou_threshold=IOU_THR)

In [12]:
for k, v in predictions.items():
    r = filtered_predictions[k]
    if len(v["boxes"]) != len(r["boxes"]):
        print(k, len(v["boxes"]), len(r["boxes"]))
        print("Boxes before NMS:", v["boxes"])
        print("Boxes after NMS:", r["boxes"])
        print("Scores before NMS:", v["scores"])
        print("Scores after NMS:", r["scores"])
        print("Labels before NMS:", v["labels"])
        print("Labels after NMS:", r["labels"])
        print()
        break

In [13]:
filtered_data = []
id_to_label = {v: k for k, v in class_id.items()}
for image_id, data in filtered_predictions.items():
    boxes = data["boxes"].tolist()
    scores = data["scores"].tolist()
    labels = data["labels"].tolist()
    for box, score, label in zip(boxes, scores, labels):
        filtered_data.append({
            "Image_ID": image_id,
            "class": id_to_label[label],
            "confidence": score,
            "ymin": box[1],
            "xmin": box[0],
            "ymax": box[3],
            "xmax": box[2],
            "class_id": label
        })

filtered_df = pd.DataFrame(filtered_data)
filtered_df.head()

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,class_id
0,ID_cWEAQI.jpeg,anthracnose,0.050378,2879.414062,935.048828,3668.637207,1365.869995,0
1,ID_cWEAQI.jpeg,healthy,0.432404,56.677937,-15.221375,3975.569824,1572.595459,2
2,ID_cWEAQI.jpeg,healthy,0.143457,322.058899,1410.376099,1089.723755,1739.138550,2
3,ID_cWEAQI.jpeg,healthy,0.088023,148.699219,13.658841,1109.639893,449.274506,2
4,ID_cWEAQI.jpeg,healthy,0.078814,14.121279,1120.934204,474.832306,1699.360962,2


In [14]:
len(filtered_df), len(test_df)

(18368, 18368)

In [15]:
filtered_df["class"].value_counts()

class
healthy        8226
cssvd          5185
anthracnose    4957
Name: count, dtype: int64

In [16]:
files = list(PREDICTION_IMAGE_PATH.glob("*"))

In [17]:
test_df: pd.DataFrame = filtered_df.copy()

In [18]:
test_records = test_df.to_dict(orient="records")

test_records[:5]

[{'Image_ID': 'ID_cWEAQI.jpeg',
  'class': 'anthracnose',
  'confidence': 0.05037793517112732,
  'ymin': 2879.4140625,
  'xmin': 935.048828125,
  'ymax': 3668.63720703125,
  'xmax': 1365.8699951171875,
  'class_id': 0},
 {'Image_ID': 'ID_cWEAQI.jpeg',
  'class': 'healthy',
  'confidence': 0.4324035346508026,
  'ymin': 56.67793655395508,
  'xmin': -15.221375465393066,
  'ymax': 3975.56982421875,
  'xmax': 1572.595458984375,
  'class_id': 2},
 {'Image_ID': 'ID_cWEAQI.jpeg',
  'class': 'healthy',
  'confidence': 0.14345668256282806,
  'ymin': 322.05889892578125,
  'xmin': 1410.3760986328125,
  'ymax': 1089.7237548828125,
  'xmax': 1739.1385498046875,
  'class_id': 2},
 {'Image_ID': 'ID_cWEAQI.jpeg',
  'class': 'healthy',
  'confidence': 0.08802306652069092,
  'ymin': 148.69921875,
  'xmin': 13.658841133117676,
  'ymax': 1109.639892578125,
  'xmax': 449.2745056152344,
  'class_id': 2},
 {'Image_ID': 'ID_cWEAQI.jpeg',
  'class': 'healthy',
  'confidence': 0.07881356030702591,
  'ymin': 14.1

In [19]:
file_id = [i.name for i in files]
file_id[:2]

['ID_cWEAQI.jpeg', 'ID_NtqErb.jpg']

In [20]:
file_okay = set(test_df["Image_ID"].values)
file_nokay = set(file_id) - file_okay

for f in file_nokay:
    test_records.append(
		{
			"Image_ID": f,
		}
	)

In [21]:
test_df = pd.DataFrame(test_records)

test_df.sample(10)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,class_id
16105,ID_qfbYv3.jpg,anthracnose,0.101511,496.230896,-9.659603,1595.764160,272.354004,0
6530,ID_tUm5mY.jpg,healthy,0.291516,2629.395752,18.211262,3253.189697,660.640686,2
6347,ID_Cy2zVL.jpg,cssvd,0.654104,28.934364,12.140007,1262.935547,589.335266,1
18046,ID_ZR2coE.jpg,healthy,0.096812,115.609085,198.383469,156.913239,267.835419,2
14573,ID_sNijYN.jpg,healthy,0.270816,1794.406128,4.282996,3192.399170,1045.734253,2
14394,ID_ORZ9Bi.jpg,healthy,0.057966,84.674576,364.492188,167.912949,408.514465,2
2822,ID_ynZOCB.jpeg,anthracnose,0.112698,3344.731689,426.577698,4034.945068,905.530273,0
13206,ID_h4waf5.jpeg,healthy,0.744963,825.986938,-4.167011,1755.193970,3402.234375,2
3336,ID_Egf8mv.jpg,healthy,0.125444,13.969364,104.084198,396.188049,395.074646,2
102,ID_fk4Rso.jpeg,healthy,0.480804,175.766449,269.264648,4017.348633,2857.361572,2


In [22]:
test_df["class"].value_counts()

class
healthy        8226
cssvd          5185
anthracnose    4957
Name: count, dtype: int64

In [23]:
test_df.to_csv(PREDICTION_PATH.replace("-prediction", "-submission"), index=False)

In [24]:
PREDICTION_PATH.replace("-prediction", "-submission")

'dataset/predictions/04-submissions.csv'